In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas.geoseries import Point

In [ ]:
#read in files
bus_to_poi = pd.read_csv("OD_allPOI_600.csv")
bus_stops = gpd.read_file('BusStop.shp')

In [ ]:
#convert csv file to geopandas dataframe
crs = {'init': 'epsg:3414'}
bus_poi_linestring = gpd.GeoDataFrame(bus_to_poi, crs=crs)

In [ ]:
bus_poi_linestring.info()

In [ ]:
#convert string geometry to geoseries
import shapely.wkt
bus_poi_linestring['geometry'] = bus_poi_linestring['geometry'].apply(shapely.wkt.loads)

In [ ]:
#separate line geometry to points for start and end points
#startpts = gpd.GeoSeries([Point(list(pt['geometry'].coords)[0]) for i, pt in bus_poi_linestring['geometry'].iterrows()])
startpts = gpd.GeoSeries([Point(list(pt['geometry'].coords)[0]) for i,pt in bus_poi_linestring[['geometry']].iterrows()])
endpts = gpd.GeoSeries([Point(list(pt['geometry'].coords)[-1]) for i,pt in bus_poi_linestring[['geometry']].iterrows()])

In [ ]:
test1 = gpd.GeoSeries([(pt['geometry'].coords)[-1] for i,pt in bus_poi_linestring[['geometry']].iterrows()])
len(test1.unique())

In [ ]:
bus_poi_linestring['geometry_stop'] = startpts
bus_poi_linestring['geometry_poi'] = endpts

In [ ]:
bus_poi_linestring.head()

In [ ]:
bus_loc = bus_poi_linestring.drop_duplicates(subset=['dest_busid'], keep='first').copy()

In [ ]:
bus_loc = gpd.GeoDataFrame(bus_loc[['dest_busid','geometry_stop']])
bus_loc.rename(columns={'geometry_stop':'geometry'}, inplace=True)

In [ ]:
bus_loc.info()

In [ ]:
bus_poi_linestring.info()

In [ ]:
#convert csv file to geopandas dataframe
#crs = {'init': 'epsg:3414'}
#bus_loc = gpd.GeoDataFrame(bus_loc, crs=crs)
#bus_loc.to_file('bus_stop_location1.shp')

In [ ]:
bus_loc.info()

In [ ]:
poi_loc = bus_poi_linestring.drop_duplicates(subset=['poi'], keep='first')

In [ ]:
#Make Point hashable to perform merge by extending the shapely.geometry.Point class
class HPoint(shapely.geometry.Point):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def __hash__self(self):
        return hash(tuple(self.coords))

In [ ]:
def dataframe_to_geodataframe(df):
    geometry = [HPoint(xy) for xy in zip(df.x, df.y)]
    gdf = gpd.GeoDataFrame(df, geometry=geometry)
    return gdf


In [ ]:
gdf = dataframe_to_geodataframe(bus_loc)

In [ ]:
bus_stops.info()

In [ ]:
merge = pd.merge(bus_loc,bus_stop_pd, left_on='geometry_stop', right_on='geometry')

In [ ]:
def getXY(pt):
    return (pt.x, pt.y)

In [ ]:
x_loc,y_loc = [list(t) for t in zip(*map(getXY, bus_loc['geometry']))] 
#x_bus,y_bus = [list(t) for t in zip(*map(getXY, bus_stops['geometry']))] 

In [ ]:
bus_loc['x'],bus_loc['y'] = x_loc,y_loc

In [ ]:
#bus_stops['x'],bus_stops['y'] = x_bus,y_bus

In [ ]:
bus_loc.sort_values(by=['x'], inplace=True)

In [ ]:
bus_stops.sort_values(by=['x'], inplace=True)

In [ ]:
bus_loc.to_file('BusStops_Location.shp')

In [ ]:
bus_stops.tail(10)

In [ ]:
bus_stop = gpd.read_file('busStop_location_svy.shp')
planning_area = gpd.read_file('planning_area.shp')

In [ ]:
busStop_pArea = gpd.sjoin(bus_stop, planning_area, op='within')
len(busStop_pArea) == len(bus_stop)

In [ ]:
busStop_pArea